In [5]:
# imports
import dash
from dash import dcc, html, Input, Output, State
import dash_cytoscape as cyto
import time


## class required


In [ ]:
# modules importants pour le(s) tp(s) :

# visualisation des couleurs et autres... (partie graphique)
from dash import Dash, html # type: ignore
import dash_cytoscape as cyto # type: ignore

# partie système
from time import time, sleep
import os, sys, random as rd
from random import randint

# partie mathématique + structure
import numpy as np # type: ignore
from math import inf, sqrt, cos, sin, tan

# ON IMPORTE LES CLASSES DU TP PRECEDENT
class V:
    '''représente les informations des sommets (vertex)'''
    def __init__(self, name:str | int, weight=0):
        self.name = name
        self.weight = weight
class Vertex:
    """
    représente les sommets et leurs adjacences dans un graphe
    
    on crée la classe qui s'occupe de toutes les adjacences entre 
    les sommets (vertex/vertices) et leurs autres sommets reliés
    les arrêtes (edges) ne sont pas représentés car c'est la matrice d'adjacence qui s'occupe de ça
    qui est situé dans la classe Graph
    """
    id = 0 # identifiant du sommet 

    def __init__(self, name:str, neighbor:list[V]=[]):
        self.name = name
        self.total_weight = sum([v.weight for v in neighbor]) # poids total du sommet
        self.neighbor = neighbor
        self.vertices_names = [v.name for v in self.neighbor]
        self.degree = len(neighbor)
        self.id = Vertex.id
        Vertex.id += 1

    def __str__(self):
        '''si on affiche Node (print(Node)) renverra ce qui suit'''
        res = f"{self.name} ["
        v_size = len(self.neighbor)
        for i, v in enumerate(self.neighbor):
            res += f"{v.name} w={v.weight}"
            if(i!=v_size-1):
                if v_size>1:
                    res+="," 
                res+=" "
        res+="]"
        return res
class Edge:
    '''représente une arrête entre deux sommets/vertex
    manière différente de représenter un graphe'''

    id = 0

    def __init__(self, vfrom:str | int, vto:str | int, weight: int | float):
        self.a = vfrom
        self.b = vto
        self.weight = weight
        self.id = Edge.id
        Edge.id += 1
    
    def __eq__(self, other:"Edge"): # equivalent to self == other
        '''permet d'utiliser l'objet dans un set (le rendre hashable)
        pour faire des comparaisons entre des ensembles (set) d'objet Edge'''
        # On considère que deux arêtes sont égales si leurs sommets et poids sont égaux
        return (self.a == other.a and self.b == other.b and self.weight == other.weight)
    
    def __ne__(self, edge:object): # equivalent de !=
        return not self.__eq__(edge)    

    def __hash__(self): # equivalent to set(self) == set(other)
        """permet d'utiliser l'objet dans un set (le rendre hashable)
        pour faire des comparaisons entre des ensembles (set) d'objet Edge"""
        # Calculer le hash en fonction des attributs de l'objet
        return hash((self.start, self.end, self.weight))

class Graph:
    """créée un graphe avec une liste de "Node" en paramètre et un titre
    @version 3.0.1"""

    def __init__(self, data:list[Vertex], title:str="default"): # OK
        '''@version 3.0.0'''
        self.title = title # titre du graphique si utilisé
        self.adj = data # liste d'adjacences
        self.edges = []
        self.verticesn = [] # liste des sommets (seulement les noms)
        self.weight = 0 # poids total du graphe    
        self.degree = 0 # pas encore calculé le degré du graphe
        self.is_complete = True
        for vx in self.adj:
            self.verticesn.append(vx.name)
            self.weight += vx.total_weight
            for v in vx.neighbor:
                self.edges.append(Edge(vx.name, v.name, v.weight))
                self.degree += 1
        self.create_matrix() # matrice des liens entre les noeuds (Edges) (matrice d'adjacences)
        self.port = 8054+randint(10,round(1e4))
        self.show_weights=True
        self.color_edges=[] # arretes a colorer (UGI)
        self.is_colored=True # autorise la coloration (UGI)

    def create_matrix(self): # OK
        """créer la matrice d'ajdacence des points\n
        (fonctionne avec les graphes orienté également)
        @version {3.0.1}
        prochaine version supprimer l'initialisation de la matrice a False
        et ajouter un par un les True et False (reduction de complexité)
        """
        # si on trouve que m[i][j] != m[j][i] 
        # ou que les poids ne sont pas les mêmes c'est oriente
        # attention dans les Vertex les V peuvent ne pas être dans l'ordre
        self.matrix = [[False for _ in self.adj] for _ in self.adj]
        self.is_oriented=False
        for lin, vis in enumerate(self.adj):
            for v in vis.neighbor:
                v_jx_idx = self.verticesn.index(v.name)
                self.matrix[lin][v_jx_idx]=True
                if not self.is_oriented:
                    if vis.name not in self.adj[v_jx_idx].vertices_names:
                        self.is_oriented=True
                    else:
                        vi_xj_idx = self.adj[v_jx_idx].vertices_names.index(vis.name)
                        vi_ji = self.adj[v_jx_idx].neighbor[vi_xj_idx]
                        if v.weight != vi_ji.weight :
                            self.is_oriented=True
        iter = range(len(self.matrix))
        # matrice d'un graphe complet avec la liste des sommets actuel
        complete = [[self.matrix[i][j] if i == j else True for i in iter] for j in iter]
        if(self.matrix == complete):
            self.is_complete = True

    def __str__(self):
        """si jamais on print un graph (print(Graph)) c'est executé ici
        affichage au plus simple du graphe avec des caractères
        @version 3.0.0"""
        res = ""
        for i, vis in enumerate(self.adj):
            res += f"{i}\t | {vis.name} ["
            v_size = len(vis.neighbor)
            for j in range(v_size):
                res += f"{vis.neighbor[j].name} w={vis.neighbor[j].weight}"
                if(j!=v_size-1):
                    if v_size>1:
                        res += "," 
                    res+=" "
            res += "]\n"
        return res
    
    def _sort_edges(self, edges:list[Edge])->list[Edge]: # OK complexite≈O(n+log(5n))
        """fonction privée a ne pas utiliser (en dehors de la classe)
        algorithme reccursif pour les problèmes de pronfondeur et de performances
        @version 3.0.0"""
        if len(edges)<2 :
            return edges
        else:
            pivot = edges[len(edges)//2].weight
            l, m, r = [],[],[] # mineurs, égal, majeurs 
            for e in edges:
                if(e.weight < pivot): l.append(e)
                    # si reccursion sur len(m) ce n'est jamais < 2
                    # et donc (boucle infini) dans certains cas                
                elif(e.weight == pivot): m.append(e) 
                else: r.append(e)
            return self._sort_edges(l)+m+self._sort_edges(r)

    def sort_by_weight(self): # OK
        """trie le graphe par poids croissants(asc)
        on trie chaque arrêtes du graphe
        @version 3.0.0
        """
        self.edges = self._sort_edges(self.edges)

    def show_edges(self):
        '''affiche les edges sous forme de liste de string
        @version 3.0.0'''
        print([e.a+e.b+" w="+str(e.weight) for e in self.edges])

    def resume(self):
        '''crée un résumé du graphe
        @version 3.0.0'''
        print("quelques informations sur le graphe : \n")
        print(f"\tdegré: {self.degree}")
        print(f"\tpoids: {self.weight}")
        print(f"\tcomplet : {self.is_complete} (def: si tous les sommets sont reliés)")
        print(f"\tplanaire: voir si les arrêtes se croisent ou non")
        print(f"\t\tnecessite surement de déplacer\n\t\tles noeuds (sur la partie graphique)")

# code de génération de couleurs selon le 
# nombre de parts (nb de couleurs nécessaire)

def generate_colors(n:int)->list[str]:
    """Génère N couleurs différentes en format hexadécimal, réparties uniformément.
    Les couleurs changent à chaque exécution grâce à un mélange aléatoire.
    """
    # Diviser l'espace des couleurs de manière uniforme
    step = 256 // n
    # Générer une palette de couleurs de base
    tmp = range(n)
    base_colors = [(i * step, j * step, k * step) for i in tmp for j in tmp for k in tmp]
    rd.shuffle(base_colors) # mélange
    selected_colors = base_colors[:n] # les N premières couleurs
    # return conveted colors to hex
    return [f"#{r:02x}{g:02x}{b:02x}" for r, g, b in selected_colors]

def int_to_rgb(color_int_:int)->str:
    '''convertit une valeur int tres grande (255**3) en rgb'''
    if type(color_int_) != int:
        raise ValueError(f"(function) int_to_rgb(n=\"{color_int_}\"), n is not int")
    color_int = color_int_ % (256**3+1)
    r = (color_int >> 16) & 0xFF
    g = (color_int >> 8) & 0xFF
    b = color_int & 0xFF
    return f"#{r:02x}{g:02x}{b:02x}"

def generate_colors2(n:int)->list[str]:
    """Génère N couleurs réparties uniformément 
    avec un décalage cyclique global aléatoire format rgb hex (#000000)"""
    max_colors = 256 ** 3  
    step = max_colors // n
    rd_decal = rd.randint(0, max_colors - 1) 
    # Générer les N couleurs en considérant l'espace continu
    colors = [(i * step + rd_decal) % max_colors for i in range(n)]
    # Convertir les couleurs décalées en format RGB puis en hexadécimal
    return [int_to_rgb(color) for color in colors]

class Graph(Graph): 
    # on ajoute des méthodes a l'existant (extension)

    def __init__(self, data, title = "default"):
        super().__init__(data, title)
        self._render()

    def _render(self):
        '''initialize graphic things'''
        self.custom_style = {
            'width': '100%', 
            'height': '500px',
            "border": "3px white solid",
            "border-radius":"5px",
            "background-color":"#666666",
            "title" : {"background-color":"white"}
        }
        self.unique = f' {time()%1e4:.5}'
        allow_arrows = "linear" # ce style n'autorise pas les flèches
        if self.is_oriented:
            allow_arrows = "bezier" # ce style oui
        self.my_styles_sheet = [{
                'selector': 'node',
                'style': {
                    'background-color': '#222222', 
                    'color': 'white',
                    'label': 'data(label)',
                    'font-size': '16px',
                    'text-valign': 'center', 
                    'text-halign': 'center' 
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'width': 2,
                    'target-arrow-shape': "vee",
                    "target-arrow-color": "#4a7cf2",
                    'arrow-scale': 2,
                    'curve-style': allow_arrows
                }
            }
        ]

    def render(self, layout_name="breadthfirst"): # rendu UGI
        """effectue le rendu du graphe visuellement
        @version 3.1.4"""
        if self.show_weights:
            self.my_styles_sheet.append({
                'selector': 'edge',
                'style': {
                    'label': 'data(weight)',
                    "color": 'white'
                }
            })   
        elems = [] # éléments à afficher (formattés)
        for node in self.adj:
            elems.append({'data': {"id":node.name, "label":node.name}})
        # add edges
        for edge in self.edges:
            elems.append({
                'data': {
                    'source': edge.a, 
                    'target': edge.b, 
                    'weight': edge.weight
                }
            })
        app = Dash(self.title+self.unique)            
        app.layout = html.Div([
            cyto.Cytoscape(
                id='cytoscape'+self.unique,
                elements=elems,
                layout={'name': layout_name},
                style=self.custom_style,
                stylesheet=self.my_styles_sheet
            )
        ])
        print('\nrendu graphique : ')
        print(f"\tégalement ouvert sur la page web : \"localhost:{self.port}\"")
        print(f"\topened too at the web page : \"localhost:{self.port}\"")
        app.run_server(debug=True,port=self.port)

    def _color_grg(self, nb_colors:int):
        '''permet de colorer le graphe affiché avec cytoscape'''
        color = generate_colors(nb_colors)
        for k in self.res["data"]:
            v = self.res["data"][k]
            self.my_styles_sheet.append(
                { # la règle la plus importante a la fin (override des précédentes)
                    'selector': f'node[id="{k}"]',
                    'style': {
                        'background-color': f'{color[v-1]}',
                    }
                }                    
            )

    def coloration(self)->dict[str:int, str:dict[str:int]]:
        """Réalise une coloration des sommets d'un graphe donné.
        renvoie un dict: Un dictionnaire associant chaque sommet à une couleur numérotée.
        @version 3.5.9
        """
        sommets_tries = sorted(self.adj, key=lambda v: len(v.neighbor), reverse=True)
        # Initialiser les couleurs
        coloration = {vertex.name: None for vertex in self.adj}
        couleur_max = 0
        # Attribuer les couleurs
        for vertex in sommets_tries:
            # Collecter les couleurs déjà utilisées par les voisins
            couleurs_voisines = [
                coloration[neighbor.name]
                for neighbor in vertex.neighbor
                if coloration[neighbor.name] is not None
            ]
            # Trouver la première couleur non utilisée
            couleur = 1
            while couleur in couleurs_voisines:
                couleur += 1
            coloration[vertex.name] = couleur
            couleur_max = max(couleur_max, couleur)
            coloration[vertex.name] = couleur
            couleur_max = max(couleur_max, couleur)
        # pour voir le résultat de l'extérieur            
        self.res=dict()
        self.res["Nombre de couleurs utilisées"]=couleur_max
        self.res["data"] = coloration
        self._color_grg(couleur_max) # rendu graphique

        

## code floyd warshal

In [ ]:
import dash
from dash import dcc, html, Input, Output, State
import dash_cytoscape as cyto
from random import randint
from math import inf
import time

# Création de l'application Dash
app = dash.Dash(__name__)

# Exemple de données de graphes (à adapter selon ta classe Vertex et V)
class V:
    def __init__(self, name, weight):
        self.name = name
        self.weight = weight

class Vertex:
    def __init__(self, name, neighbor):
        self.name = name
        self.neighbor = neighbor
        self.total_weight = sum([v.weight for v in neighbor])

class Edge:
    def __init__(self, source, target, weight):
        self.source = source
        self.target = target
        self.weight = weight

class Graph(Graph):

    """Créée un graphe avec une liste de "Vertex" en paramètre"""
    def __init__(self, data:list[Vertex], title:str="default"):
        super.__init__
        self.__steps = []  # Liste pour stocker les étapes du calcul

    def create_matrix(self):
        """Créer la matrice d'adjacence"""
        self.matrix = [[float('inf')] * len(self.adj) for _ in range(len(self.adj))]
        for i, v in enumerate(self.adj):
            self.matrix[i][i] = 0
            for neighbor in v.neighbor:
                j = self.verticesn.index(neighbor.name)
                self.matrix[i][j] = neighbor.weight

    def _floyd_warshall_grg(self):
        '''Met à jour l'affichage graphique avec les étapes du calcul'''
        # A chaque étape de calcul, enregistre les étapes
        steps = []
        length = len(self.adj)
        # Mise à jour graphique à chaque étape
        for k in range(length):
            for i in range(length):
                for j in range(length):
                    steps.append((i, k, j, self.matrix[i][j]))  # Enregistre avant la mise à jour
                    self.matrix[i][j] = min(self.matrix[i][j], self.matrix[i][k] + self.matrix[k][j])
                    steps.append((i, k, j, self.matrix[i][j]))  # Enregistre après la mise à jour
        self.__steps = steps  # Stocke toutes les étapes

    def floyd_warshall(self) -> list[list[float]]:
        '''Calcul des chemins optimaux entre toutes les paires de sommets et renvoie une matrice des résultats'''
        # Initialisation de la matrice des distances
        length = len(self.adj)
        index_map = {v.name: idx for idx, v in enumerate(self.adj)}  # Mapping noms -> indices
        # Initialisation des distances
        res = [[inf] * length for _ in range(length)]
        for i, v in enumerate(self.adj):
            res[i][i] = 0
            for neighbor in v.neighbor:
                vj = index_map[neighbor.name]
                res[i][vj] = neighbor.weight  # Distance initiale entre voisins
        length = range(length)
        # L'algorithme Floyd-Warshall avec mise à jour graphique
        for k in length:  # Sommets intermédiaires
            for i in length:  # Source
                for j in length:  # Destination
                    res[i][j] = min(res[i][j], res[i][k] + res[k][j])
            # Mise à jour graphique à chaque itération de l'algorithme
            self._floyd_warshall_grg()  # Appelle la mise à jour graphique
        return res  # Retourne la matrice des distances finales


# Instantiation du graphe
data = [
    Vertex("A", [V("B", 2), V("C", 4), V("D", 6)]),
    Vertex("B", [V("C", 1), V("F", 7)]),
    Vertex("C", [V("E", 1)]),
    Vertex("D", [V("F", 6)]),
    Vertex("E", [V("F", 1)]),
    Vertex("F", [])
]
graph_instance = Graph(data)

# Callback pour démarrer le calcul et afficher les étapes dans Dash
@app.callback(
    Output('store-data', 'data'),
    Input('start-button', 'n_clicks'),
    State('store-data', 'data')
)
def start_floyd_warshall(n_clicks, stored_data):
    if n_clicks is None:
        raise dash.exceptions.PreventUpdate

    # Démarre l'algorithme et stocke les étapes dans self.__steps
    steps = graph_instance.floyd_warshall()  # Exécute l'algorithme
    return graph_instance.__steps  # Retourne les étapes calculées


# Callback pour afficher les étapes dans Cytoscape
@app.callback(
    Output('cytoscape', 'elements'),
    Input('store-data', 'data')
)
def update_graph(data):
    if not data:
        raise dash.exceptions.PreventUpdate

    # Pour chaque étape de calcul, mets à jour les éléments du graphe (Cytoscape)
    updated_elements = []
    for step in data:
        i, k, j, distance = step
        # Crée une nouvelle arête ou mets à jour une arête avec la distance
        updated_elements.append({
            'data': {
                'id': f"edge-{i}-{j}",
                'source': f"node-{i}",
                'target': f"node-{j}",
                'weight': distance
            }
        })
    return updated_elements

# Layout de l'application
app.layout = html.Div([
    dcc.Store(id='store-data'),  # Pour stocker l'état des étapes
    cyto.Cytoscape(
        id='cytoscape',
        elements=[{'data': {'id': 'A'}}, {'data': {'id': 'B'}}, {'data': {'id': 'C'}}],  # Exemple
        style={'width': '600px', 'height': '400px'},
        layout={'name': 'circle'}
    ),
    html.Button("Start Calculation", id="start-button"),
    html.Div(id='output')
])

if __name__ == '__main__':
    app.run_server(debug=True,post=9099)


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[16], line 129, in start_floyd_warshall(n_clicks=1, stored_data=None)
    127 # Démarre l'algorithme et stocke les étapes dans self.__steps
    128 steps = graph_instance.floyd_warshall()  # Exécute l'algorithme
--> 129 return graph_instance.__steps  # Retourne les étapes calculées
        graph_instance = <__main__.Graph object at 0x12460c590>

AttributeError: 'Graph' object has no attribute '__steps'

